In [1]:

from enhancing.modules.stage1.vitvqgan import ViTVQ
from enhancing.modules.stage1.layers import ViTEncoder as Encoder, ViTDecoder as Decoder
# from enhancing.modules.stage1.quantizers import VectorQuantizer, GumbelQuantizer
from enhancing.utils.general import get_config_from_file, initialize_from_config

from taming.models.vqgan import VQModel

import torch

/home/leonard/.local/lib/python3.10/site-packages/wandb/sdk/internal/internal_api.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [2]:

config = get_config_from_file("/home/leonard/semcom_ws/src/commun/commun/vqgan_f16_16384.yaml")
vitvq: VQModel = initialize_from_config(config.model)
vitvq.init_from_ckpt("/home/leonard/semcom_ws/src/commun/commun/checkpoint/vqgan_f16_16384.ckpt")
for param in vitvq.parameters():
    param.requires_grad = False
vitvq.eval()
# vitvq.cuda()
print()

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
Restored from /home/leonard/semcom_ws/src/commun/commun/checkpoint/vqgan_f16_16384.ckpt



In [ ]:

for b in [1]: #, 2, 3, 4, 6]:
    dummy_input = torch.randn(b, 1024, 512)
    torch.onnx.export(vitvq.decoder, dummy_input, f"./vqgan_{b}.onnx", export_params=True)
    print(f"ONNX model saved: vqgan_{b}.onnx")

[torch.onnx] Obtain model graph for `ViTDecoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ViTDecoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 8 of general pattern rewrite rules.
ONNX model saved: decoder_small_1.onnx
[torch.onnx] Obtain model graph for `ViTDecoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ViTDecoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 8 of general pattern rewrite rules.
ONNX model saved: decoder_small_2.onnx
[torch.onnx] Obtain model graph for `ViTDecoder([...]` with `torch.export.export(..., 

In [ ]:
# trtexec --onnx=decoder_base_1.onnx --fp16